## Volbrain deeplesion verzamelen

This code is for organizing the Volbrain.net output of the DeepLesion Algorythm. It runs over the zip files in a specific folder, and extracts necessary information from the enclosed CSVs and puts themm all together in a table

In [1]:
#load necessary libraries
import pandas as pd
import numpy as np 
import zipfile
import os
from zipfile import ZipFile
import glob

In [2]:
#navigate to zipfolders and intended folder for output
path_volbrain = 'Z:/VolBrain'
output_folder = "Z:/VolBrain/Separate_CSV_Deeplesion"


In [3]:
#create a list to store the CSV headers in
all_data = []

In [4]:
dataframes = []
from io import BytesIO


for zip_file in glob.glob(os.path.join(path_volbrain, '*.zip')):
    # split by underscores
    participant_id = zip_file.split("_")
    # concatenate BRICK with 00X
    participant_id = participant_id[1] + "_" + participant_id[2]
    with ZipFile(zip_file, "r") as zf:
        #get the list of files inside the zip (assumes 1 csv per zip folder)
        for entry_name in zf.namelist():
            if entry_name.endswith(".csv"):
                entry = zf.read(entry_name)
                df = pd.read_csv(BytesIO(entry), sep=';')
                df['Paricipant Id'] = participant_id
                dataframes.append(df)

In [5]:
concat_df = pd.concat(dataframes)
concat_df

,Subject,Sex,Age,Report date,Quality control T1,Quality control FLAIR,White Matter (WM) volume cm3,White Matter (WM) volume %,Normal appearing WM volume cm3,Normal appearing WM volume %,...,Thalamic radiation anterior right disconnection probability,Thalamic radiation posterior left disconnection probability,Thalamic radiation posterior right disconnection probability,Thalamic radiation superior left disconnection probability,Thalamic radiation superior right disconnection probability,Uncinate fasciculus left disconnection probability,Uncinate fasciculus right disconnection probability,Vertical occipital fasciculus left disconnection probability,Vertical occipital fasciculus right disconnection probability,Paricipant Id
0,job1729059,Female,16.0,10-Sep-2024,A,A,426.365775,31.822560,425.491947,31.757340,...,0.000000,70.046951,71.912365,61.019451,70.375589,0.000000,0.000000,51.643193,68.024312,BRICK_001
0,job1729067,Male,15.0,10-Sep-2024,A,A,400.018627,33.516918,399.391610,33.464381,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,BRICK_002
0,job1729071,Female,13.0,10-Sep-2024,A,B,460.715082,31.271578,458.996232,31.154909,...,0.000000,74.674181,63.192490,61.391380,67.839608,0.000000,0.000000,56.619720,0.000000,BRICK_003
0,job1729073,Male,9.0,10-Sep-2024,A,B,455.632899,31.341535,453.964613,31.226779,...,82.339908,50.140846,0.000000,64.734830,68.595711,55.993742,50.516433,0.000000,0.000000,BRICK_004
0,job1729088,Male,9.0,10-Sep-2024,A,B,417.806187,30.485584,417.656813,30.474685,...,0.000000,0.000000,0.000000,0.000000,72.508677,0.000000,0.000000,0.000000,0.000000,BRICK_005
0,job1729376,Male,9.0,11-Sep-2024,A,B,434.294205,31.854803,433.575530,31.802090,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,BRICK_006
0,job1729091,Female,14.0,10-Sep-2024,A,A,378.981688,32.289125,378.248562,32.226663,...,70.249569,0.000000,55.577466,55.613684,64.225355,50.704229,55.070424,0.000000,0.000000,BRICK_007
0,job1729092,Male,17.0,10-Sep-2024,A,A,460.770814,31.860121,459.173720,31.749690,...,64.933191,0.000000,56.951711,67.498637,69.885059,0.000000,0.000000,0.000000,70.324558,BRICK_008
0,job1729093,Male,6.0,10-Sep-2024,A,C,443.976699,32.419402,423.977238,30.959031,...,74.608780,67.054474,90.187796,66.361504,71.773830,59.473966,63.779268,74.924519,59.300852,BRICK_009
0,job1729095,Female,11.0,10-Sep-2024,A,B,433.407448,29.697125,432.829845,29.657547,...,0.000000,77.757461,77.737267,66.485426,0.000000,0.000000,0.000000,0.000000,0.000000,BRICK_010


In [6]:
concat_df.to_csv('Z:/Aida_experiment/volbrainsnew.csv')